In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
def get_data_loader(training = True):
    custom_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_set = datasets.FashionMNIST('./data', train = True, download=True, transform=custom_transform)
    test_set = datasets.FashionMNIST('./data', train = False, transform=custom_transform)
    
    if training:
        loader = torch.utils.data.DataLoader(train_set, batch_size = 64)
    else:
        loader = torch.utils.data.DataLoader(test_set, batch_size = 64)
    return loader

In [3]:
train_loader = get_data_loader()
print(type(train_loader))
print(train_loader.dataset)
test_loader = get_data_loader(False)

<class 'torch.utils.data.dataloader.DataLoader'>
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [4]:
def build_model():
    model = nn.Sequential(nn.Flatten(), nn.Linear(28*28, 128), nn.ReLU(), nn.Linear(128, 64), nn.ReLU(), nn.Linear(64, 10))
    return model

In [5]:
model = build_model()
print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
def train_model(model, train_loader, criterion, T):
    """
    TODO: implement this function.

    INPUT: 
        model - the model produced by the previous function
        train_loader  - the train DataLoader produced by the first function
        criterion   - cross-entropy 
        T - number of epochs for training

    RETURNS:
        None
    """

    # set the model to train mode before iterating
    model.train()
    counter = 0
    # outer for loop: iterates through epochs
    for epoch in range(T):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        
        # inner for loop: iterates through (images, labels) pairs from the train_loader
        for (images, labels) in train_loader:
            opt.zero_grad()

            # forward + backward + optimize
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            opt.step()

            running_loss += loss.item()
            total_predictions += labels.size(0)
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability (ChatGPT helped me with this part)
            correct_predictions += (predicted == labels).sum().item()  # Update correct count
            accuracy_percent = round((correct_predictions / total_predictions) * 100, 2)
        # printing everything
        print(f'Train Epoch: {counter}   Accuracy: {correct_predictions}/{total_predictions}({accuracy_percent}%)   Loss: {round(running_loss / len(train_loader), 3)}')
        counter += 1


In [7]:
criterion = nn.CrossEntropyLoss()
train_model(model, train_loader, criterion, 5)

Train Epoch: 0   Accuracy: 42858/60000(71.43%)   Loss: 0.883
Train Epoch: 1   Accuracy: 49274/60000(82.12%)   Loss: 0.509
Train Epoch: 2   Accuracy: 50345/60000(83.91%)   Loss: 0.455
Train Epoch: 3   Accuracy: 51006/60000(85.01%)   Loss: 0.423
Train Epoch: 4   Accuracy: 51516/60000(85.86%)   Loss: 0.401


In [8]:
def evaluate_model(model, test_loader, criterion, show_loss = True):
    correct = 0
    total = 0
    running_loss = 0.0
    model.eval()
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            
            # run images through the model to calculate the outputs
            outputs = model(images)
            
            # we want the class with the highest energy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # calculate the loss and update the total
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        accuracy = (correct / total) * 100
        if (show_loss == True):
            print(f'Average Loss: {round(running_loss / len(test_loader), 4)}')
        print(f'Accuracy: {round(accuracy, 2)}%')

In [9]:
evaluate_model(model, test_loader, criterion, show_loss = True)

Average Loss: 0.4281
Accuracy: 84.49%


In [10]:
def predict_label(model, test_images, index):
    # find logits
    logits = model(test_images[index])
    prob = F.softmax(logits, dim=1)
    
    # assumed class names
    class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']

    index = 0
    sorted_with_index = []
    for p in prob[0]:
        sorted_with_index.append([p.item(), index])
        index = index + 1
    sorted_with_index = sorted(sorted_with_index, reverse = True)
    top1 = sorted_with_index[0]
    top1_label = class_names[top1[1]]
    top2 = sorted_with_index[1]
    top2_label = class_names[top2[1]]
    top3 = sorted_with_index[2]
    top3_label = class_names[top3[1]]
    print(f'{top1_label}: {round(top1[0] * 100, 2)}%')
    print(f'{top2_label}: {round(top2[0] * 100, 2)}%')
    print(f'{top3_label}: {round(top3[0] * 100, 2)}%')

In [11]:
test_images = next(iter(test_loader))[0]
predict_label(model, test_images, 1)

Pullover: 94.19%
Shirt: 4.78%
Coat: 0.98%
